# RV calculation at Observation Times.

To determine the expected RV shifts of the spectral lines seen my stellar subracted spectra.

In [ ]:
from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt

from PyAstronomy import pyasl   # for doppler shift
import sys
#sys.path.append('/home/jneal/azores/planet/rvs/exonailer/utilities')
try: 
    #import ajplanet
    from ajplanet import pl_rv_array
except:
    pass
%pylab inline

In [ ]:
#Rodrigo Diaz's true anomaly calculation
def trueanomaly(ma, ecc, niterationmax=10000):
    """
    Compute the true anomaly using the Newton-Raphson method.

    :param array-like ma: mean anomaly.
    :param float ecc: orbital eccentricity.
    :param int niterationmax: maximum number of iterations for N-R method.
    """
    
    if not isinstance(ma, float):
        ea = ma
    else:
        ea = np.array([ma,])

    # Initialise at ea0 = ma
    niteration = 0
    ea0 = ma
    
    while np.linalg.norm(ea - ea0, ord=1) > 1e-5 or niteration==0:
        ea0 = ea
 
        ff = ea - ecc*np.sin(ea) - ma   # Function
        dff = 1 - ecc*np.cos(ea)        # Derivative

        # Use Newton method
        ea = ea0 - ff / dff

        # Increase iteration number; if above limit, break with exception.
        niteration += 1
        if niteration >= niterationmax:
            raise RuntimeError('Eccentric anomaly computation'
                               'not converged.')
        
    # Compute true anomaly from eccentric anomaly
    return 2. * np.arctan2(np.sqrt(1. + ecc) * np.sin(ea/2.),
                           np.sqrt(1. - ecc) * np.cos(ea/2.))

# Mean Anomaly calculation
def meananomaly(t, T0, P):
    """ Calculate mean Anomaly using period, tau and a time value"""
    if not isinstance(t, np.ndarray):
        t = np.array(t)
    return 2 * np.pi * (t - T0) / P

# Basic RV calculation
def radialvelocity(gamma, K, ta, omega, ecc):
    # Calculate radial velocity of star
    return gamma + K *(np.cos(ta + omega) + ecc * np.cos(omega))

# RV calculation done in python (for when ajplanet is not available)
def rv_curve_py(t, gamma, K, omega, ecc, T, P):
    ma = meananomaly(t, T, P)
    ta = trueanomaly(ma, ecc)
    rv = radialvelocity(gamma, K, ta, omega, ecc)
    return rv
        

In [ ]:
#Silly epoch of T0 in a paper of 1997.04 +- 0.02 . need to turn into julian days
#Crepp 2016 updated HD4747 parameters  t = 1997.04
#print(0.04 * 365)
#print(.6*24)
#print(.4*60)
# Julian date for CE  1997 January 14 14:24:00.0 UT is JD 2450463.100000

# Orbtial Parameters and epochs.

In [ ]:

#Target parameters:  Obtained from sahlmann et al. 

#HD4747_params   = [9.904, 703.3, -94.2, 0.723, 62059.1, 11593.2] # Best fit solution from sahlmann not complete period coverage
#Crepp 2016 updated HD4747 parameters  t = 1997.04
#                 [mean_val, K1, omega,   e,     Tau,       Period, starmass (Msun), msini(Mjup), i] 
d = 365.25  # Days in a year
HD4747_params   = [-0.219,  755.3,  -69.1, 0.740,  50463.10,  37.88*d, 0.81, 46.1]  # turn 37.88 years into days
HD162020_params = [-27.328,  1813,  28.40, 0.277, 51990.677, 8.428198, 0.74, 14.4]  # here mean val in km/s K in m/s https://arxiv.org/pdf/astro-ph/0202458v2.pdf
HD167665_params = [8.003,   609.5, -134.3, 0.340,   56987.6,   4451.8, 1.14, 50.6]  # sahlman
HD211847_params = [6.689,   291.4,  159.2, 0.685,   62030.1,   7929.4, 0.94, 19.2]  # Best fit solution from sahlmann not complete period coverage
HD30501_params =  [23.710, 1703.1,   70.4, 0.741,   53851.5,   2073.6, 0.81, 90]  # sahlman


# 2 companions
# One of these used 3 body equations - need to compare relative strengths
HD168443b_params = [-0.046533, 475.133, 172.923, 0.52883, 15626.199, 58.1125, 0.995, 7.659] #  % msini = 7.659Mjp, a=0.2931AU, dv/dt (ms−1 yr−1) −0.00868
HD168443c_params = [-0.046533, 297.70, 64.87, 0.2113, 15521.3, 1749.83, 0.995, 17.193] 

HD202206b_params = [14.721, 564.75, 161.18, 0.435, 52250.00, 255.87, 1.04, 17.4] #\citet{Correia2005_hd202206bc}\\  % \lambda = 266.228 deg (mean longitude)
HD202206c_params = [14.721, 42.01, 78.9, 0.267, 52250.00, 1383.4, 1.04, 2.44] # \citet{Correia2005_hd202206bc}\\ % \lambda = 30.586 deg (mean longitude)

HD30501_mass = [0.81, 90]  # acutal
HD168443_massb = [0.995, 7.659]
HD168443_massc = [0.995, 17.193] # minimum i 
HD4747_mass = [0.81, 46.1]
HD167665_mass = [1.14, 50.6]  # minimum
HD211847_mass = [0.94, 19.2]
HD162020_mass = [0.74, 14.4]  # minimum
HD202206_massb = [1.04, 17.4]  # minimum
HD202206_massc = [1.04, 2.44]
#Median times 
# extremeties are +- 14 minutes
HD30501_times =  [2456024.505902, 2456140.88716, 2456141.86633, 2456145.904258]  # from data - need to double check with calculated data centers
HD162020_times = [2456112.76624, 2456112.79015]
HD202206_times = [ 2456120.78801, 2456121.73727, 2456119.85411]   
HD211847_times = [2456114.8035, 2456121.78793]  
HD4747_times =   [2456114.81674]
HD167665_times = [2456136.70895, 2456136.73434, 2456144.62087] 
HD168443_times = [2456144.68718, 2456144.70753]
params_dict = {"HD30501":HD30501_params, "HD211847":HD211847_params, \
               "HD4747":HD4747_params, "HD167665":HD167665_params, \
               "HD162020":HD162020_params, \
               "HD202206b":HD202206b_params,"HD202206c":HD202206c_params, \
               "HD168443b":HD168443b_params,"HD168443c":HD168443c_params}
               # "HD202206":HD202206_params,
    
times_dict = {"HD30501":HD30501_times, "HD162020":HD162020_times, \
              "HD202206":HD202206_times, "HD211847":HD211847_times, \
              "HD4747":HD4747_times, "HD167665":HD167665_times, \
             "HD168443":HD168443_times}


# Keplerian Orbital Parameters:

In [ ]:
def RV_from_params(t, params, use_offset=True):
    """ Get RV values with parameter list.
    
    input:
        t -- The time/s at which to calculate the RV value
        params -- a list of values [mean_val, K1, omega, e, Tau, Period]
    
    omega should be given in degrees. This function converts it to radians.
    
    Outputs:
        RVs -- The radial velocity values evaluated at the given times.
    
    """
    if not isinstance(t, np.ndarray):
        t = np.array(t)
    
    #Solar Mass conversion to MJup
    Msun_Mjup = 1047.56 # Mjup
    
    #RV for star
    star_params = params[:]
    planet_params = params[:]
    # Note: test that is np.deg2rad is faster than *np.pi /180
    star_params[2] = np.deg2rad(star_params[2])
    planet_params[2] = np.deg2rad(planet_params[2])
        
    if not use_offset:
        star_params[0] = 0
        planet_params[0] = 0
    else:
        star_params[0] = star_params[0] * 1000 # Turn into m/s
        planet_params[0] = planet_params[0] * 1000 # Turn into m/s
   
    # Scale K for planet
    K1 = planet_params[1]
    mass_ratio = Msun_Mjup*planet_params[6] / planet_params[7]
    #print("star/planet Mass Ratio", mass_ratio)
    K2 = -mass_ratio * K1
    planet_params[1] = K2
    
    #planet_params[2] += np.pi # omeaga phase do I need to add pi### NO!
    
    try: # Try rv_curve first
        #import ajplanet
        star_rvs = pl_rv_array(t, *star_params[0:6]) # *unpacks parameters from list
        planet_rvs = pl_rv_array(t, *planet_params[0:6]) # *unpacks parameters from list
        #print("used ajplanet")
    except:
        star_rvs = rv_curve_py(t, *star_params[0:6]) # *unpacks parameters from list
        planet_rvs = rv_curve_py(t, *planet_params[0:6]) # *unpacks parameters from list
        #print("used python")
        
    return star_rvs, planet_rvs


def plot_RV_phase_curve(params, name=False, t_vals=False, use_offset=False):
    """Plot RV phase curve use period and T0 to get times from phase.
    
    params -- a list of values [mean_val, K1, omega, e, Tau, Period]"""
    
    phase = np.linspace(-0.1,1.1, 200)
    t = params[4] + phase * params[5]
    
    star_rvs, planet_rvs = RV_from_params(t, params, use_offset=use_offset)
    
    plt.figure()
    plt.plot(phase, star_rvs, label="Star", lw=2)
    plt.plot(phase, planet_rvs/100, label="Planet/100", lw=2)
    if name:
        plt.title("RV Phase curve for {}".format(name))
    else:
        plt.title("RV Phase curve")
    if t_vals:
        for t_num, t_val in enumerate(t_vals):
            phi = (t_val - params[4])/params[5]  % 1
            rv_star, rv_planet = RV_from_params(t_val, params, use_offset=False)
            plt.plot(phi, rv_star, ".", markersize=12, markeredgewidth=3)
            plt.plot(phi, rv_planet/100, ".", markersize=12, markeredgewidth=3)
            plt.legend(loc=0)
    plt.xlabel("Phase")
    plt.ylabel("RV amplitude (m/s)")
    plt.show()
    

def plot_RV_curve_section(times, params, name=False, use_offset=False):
    """Plot RV phase curve use period and T0 to get times from phase.
    
    params -- a list of values [mean_val, K1, omega, e, Tau, Period]"""
    
    time_diff = np.max(times) - np.min(times)
    dt = np.max([0.05 * time_diff, 1/24.])
    #Need to find a good choice for t span
    t = np.linspace(np.min(times)-dt, np.max(times)+dt, 100)
    #phase = np.linspace(-0.1,1.1, 100)
    #t = params[4] + phase * params[5]
    
    star_rvs, planet_rvs = RV_from_params(t, params, use_offset=use_offset)
    
    plt.figure()
    plt.plot(t, star_rvs - star_rvs[0], label="Star - {0}m/s".format(int(star_rvs[0])))
    plt.plot(t, (planet_rvs-planet_rvs[0])/100, label="(Planet-{0}m/s)/100".format(int(planet_rvs[0])))
    if name:
        plt.title("RV difference curve from t[0] for {}".format(name))
    else:
        plt.title("RV difference curve from from t[0]")
    
    obs_time_sigma = 14/(60*24)  # +-14min
    
    for t_num, t_val in enumerate(times):
            rv_star, rv_planet = RV_from_params(t_val, params, use_offset=False)
            
            t_obs = np.linspace(t_val-obs_time_sigma, t_val+obs_time_sigma, 100)
            star_obs, planet_obs = RV_from_params(t_obs, params, use_offset=False)
            
            plt.plot(t_obs, star_obs-star_rvs[0], lw=3)
            plt.plot(t_obs, (planet_obs-planet_rvs[0])/100, lw=3)
            plt.plot(t_val, rv_star-star_rvs[0], ".", markersize=12, markeredgewidth=2)
            plt.plot(t_val, (rv_planet - planet_rvs[0])/100, ".", markersize=12, markeredgewidth=2)
    plt.legend(loc=0)
    plt.xlabel("Time (JD)")
    plt.ylabel("delta RV (m/s)")
    plt.show()    

    
def test_rv_curves(t, params):
    
    if not isinstance(t, np.ndarray):
        t = np.array(t)
    else:
        pass
    params = params[:]
    params[2] = np.deg2rad(params[2])
    params[0] = 0

    aj_rv = pl_rv_array(t, *params) # *unpacks parameters from list
    
    py_rv = rv_curve_py(t, *params)
    
    print("Ajplanet RVs = {}".format(aj_rv))
    print("Python RVs   = {}".format(py_rv))
    return None


def Obs_RV_error(t, params, name=False ):
    
    obs_time_sigma = 14/(60*24)  # +-14min
    
    if not isinstance(t, np.ndarray):
        t = np.array(t)
    
    t_start = t - obs_time_sigma
    t_end =  t + obs_time_sigma
    
    star_rv_start, planet_rv_start = RV_from_params(t_start, params, use_offset=False)
    star_rv_end, planet_rv_end = RV_from_params(t_end, params, use_offset=False)
    
    star_diff = star_rv_end - star_rv_start 
    planet_diff = planet_rv_end - planet_rv_start
    #if name:
        #print("RV change over observation for {} m/s".format(RV_difference))
    return star_diff, planet_diff


def RV_calculations(times, params, use_offset=False):
    Msun_Mjup = 1047.56 # Mjup
    
    for target in times:
        print("\nTarget {}".format(target))
        target_time = times[target]
        
        if target in ["HD202206", "HD168443"]:
            targetb = target + "b" 
            targetc = target + "c"
            target_paramb = params[targetb]
            target_paramc = params[targetc]
            
            star_rv_values, planet_rv_values = RV_from_params(target_time, target_paramb, use_offset=use_offset)
            star_rv_values_c, planet_rv_values_c = RV_from_params(target_time, target_paramc, use_offset=use_offset)
            star_rv_diff_b, planet_rv_diff_b = Obs_RV_error(target_time, target_paramb, name=targetb)
            star_rv_diff_c, planet_rv_diff_c = Obs_RV_error(target_time, target_paramc, name=targetc)
            print("Radial velocity values of the host star.")
            print("Gamma subtracted RV values for b = {0} m/s".format(star_rv_values))
            print("RV change over exptime for b     = {0} m/s".format(star_rv_diff_b))
            print("Gamma subtracted RV values for c = {0} m/s".format(star_rv_values_c))
            print("RV change over exptime for c     = {0} m/s".format(star_rv_diff_c))
            print("Combined RV value for triplet    = {0} m/s".format(star_rv_values + star_rv_values_c))
            print("Combined RV Diff between Obs     = {0} m/s".format(np.diff(star_rv_values + star_rv_values_c)))
            
            print("\nRadial velocity values of the planet.")
            print("Maximum mass ratio value for b   = {0} ".format(Msun_Mjup*target_paramb[6]/target_paramb[7]))
            print("Maximum mass ratio value for c   = {0} ".format(Msun_Mjup*target_paramc[6]/target_paramc[7]))
            print("Gamma subtracted RV values for b = {0} m/s".format(planet_rv_values))
            print("RV change over exptime for b     = {0} m/s".format(planet_rv_diff_b))
            print("Gamma subtracted RV values for c = {0} m/s".format(planet_rv_values_c))
            print("RV change over exptime for c     = {0} m/s".format(planet_rv_diff_c))
            print("Combined RV value for triplet    = {0} m/s".format(planet_rv_values + planet_rv_values_c))
            print("Combined RV Diff between Obs     = {0} m/s".format(np.diff(planet_rv_values + planet_rv_values_c)))
            
            star_successive_diff = np.diff(star_rv_values)
            print("Successive Star RV Diff of b            = {0} m/s".format(star_successive_diff))
            planet_successive_diff = np.diff(planet_rv_values)
            print("Successive planet RV Diff of b          = {0} m/s".format(planet_successive_diff))
            
            star_low_doppler, star_high_doppler = band_doppler_shifts(np.abs(star_successive_diff))
            print("Band Lower wl shift of b - star         = {} nm".format(star_low_doppler))
            print("Band Upper wl shift of b - star         = {} nm".format(star_high_doppler))
            planet_low_doppler,  planet_high_doppler = band_doppler_shifts(np.abs(planet_successive_diff))
            print("Band Lower wl shift of b - planet       = {} nm".format(planet_low_doppler))
            print("Band Upper wl shift of b - planet       = {} nm".format(planet_high_doppler))
            
            plot_RV_phase_curve(target_paramb, name=targetb, t_vals=target_time, use_offset=False)
            plot_RV_phase_curve(target_paramc, name=targetc, t_vals=target_time, use_offset=False)
            
            plot_RV_curve_section(target_time, target_paramb, name=targetb, use_offset=False)
            plot_RV_curve_section(target_time, target_paramc, name=targetc, use_offset=False)
      
        else:
            target_param = params[target]   
                                
            star_rv_values, planet_rv_values = RV_from_params(target_time, target_param, use_offset=use_offset)
            star_rv_diff, planet_rv_diff = Obs_RV_error(target_time, target_param, name=target)
            print("Radial velocity values of the host star only currently.")
            
            print("Gamma subtracted rv values = {0} m/s".format(star_rv_values))
            print("RV change over exptime     = {0} m/s".format(star_rv_diff))
            
            print("Radial velocity values of the Planet.")
            print("Maximum mass ratio value   = {0} ".format(Msun_Mjup*target_param[6]/target_param[7]))
            print("Gamma subtracted rv values = {0} m/s".format(planet_rv_values))
            print("RV change over exptime     = {0} m/s".format(planet_rv_diff))
            # Differences in RV between successive observations
            
            star_successive_diff = np.diff(star_rv_values)
            print("Successive Star RV Diff - star     = {0} m/s".format(star_successive_diff))
            planet_successive_diff = np.diff(planet_rv_values)
            print("Successive planet RV Diff - planet = {0} m/s".format(planet_successive_diff))
            
            star_low_doppler, star_high_doppler = band_doppler_shifts(np.abs(star_successive_diff))
            print("Band Lower wl shift - star         = {} nm".format(star_low_doppler))
            print("Band Upper wl shift - star         = {} nm".format(star_high_doppler))
            planet_low_doppler,  planet_high_doppler = band_doppler_shifts(np.abs(planet_successive_diff))
            print("Band Lower wl shift - planet       = {} nm".format(planet_low_doppler))
            print("Band Upper wl shift - planet       = {} nm".format(planet_high_doppler))
            
            
            #print("\n\nTesting Values\n")
            #print("planet/Star rv_values ratio  ", planet_rv_values/star_rv_values)
            #print("planet/Star rv_values ratio  ", planet_rv_diff/star_rv_diff)
            #print("planet/Star successive diff ratio ", planet_successive_diff/star_successive_diff)
            #print("planet/Star low doppler shift ratio  ", planet_low_doppler/star_low_doppler)
            #print("planet/Star high doppler shift ratio ", planet_high_doppler/star_high_doppler)
            
            plot_RV_phase_curve(target_param, name=target, t_vals=target_time, use_offset=False)
            
            plot_RV_curve_section(target_time, target_param, name=target, use_offset=False)
   
    return None

def band_doppler_shifts(RVs):
    """ Calculate wavelenght shift at start and end of observation wavelenght range
    
    To find expected waveleght shifts to look for.
    
    """
    lower_org_wl = 2121.056  #nm
    upper_org_wl = 2160.205  #nm
    #middle_wl = (lower_wl + upper_wl)/2.0
    #wavelengths = np.array([lower_wl, middle_wl,upper_wl])
    wavelengths = np.array([lower_org_wl, upper_org_wl]) * 10
    upper_wls = np.empty_like(RVs)
    lower_wls = np.empty_like(RVs)
    #print(wavelengths)
    for i, rv in enumerate(RVs):
        rv /= 1000  # turn into km/s
        __ , new_wls = pyasl.dopplerShift(wavelengths, np.ones_like(wavelengths), rv, edgeHandling="firstlast")
        #print(i, new_wls)
        #result_wls.append(new_wls - wavelengths)
        lower_wls[i] = new_wls[0] / 10.
        upper_wls[i] = new_wls[1] / 10.
    lower_shifts = lower_wls - lower_org_wl
    upper_shifts = upper_wls - upper_org_wl
    return lower_shifts, upper_shifts




### Run analysis for all targets:

In [ ]:
print("For ploting the RV curves the Planet RV has been divided by a factor of 100.")
print("When zooming up to just during the observations. The values are the difference from the RV at the start of this time space. To show the relative diferences, The planet RV is again scaled down by a factor of 100.")
print("The tick lines visible in a couple of the plots show the RV change during observation nod cycle. Only visible when observations were very close.")

RV_calculations(times_dict, params_dict)


From testing it appears that all the calculations for the planet RV, expected doppler shifts, RV difference between observations are all just scaled by the planet to star mass ratio m1/m2.

When only a minimum planet mass is given (m2sinI) then this corresponds to a maxmium of the mass ratio and therefore this will be a maximum expected doppler shift.



### Test for planet RV (old)
Need to determine how to calculate the expected RV of the planet (not the star) to be able to get proper values out of this. If the planet RV is > 100m/s it would be good for visible detections i think.


In [ ]:
## Calcualte K of planet

def RV_planet_old(time, star_params, masses, only_minimum=False, use_offset=True):
    """ Calcualte RV value for a planet instead of star
    
    Need to change the K value"""
    star_mass = masses[0]
    planet_mass = masses[1]
    
    planet_params = star_params[:]
    
    planet_params[1] = -(star_mass/planet_mass) * planet_params[1] 
    #planet_params[2] = planet_params[2] + 180    # phase offset
    
    planet_rv = RV_from_params(time, planet_params, use_offset=use_offset)
    
    return planet_rv


In [ ]:
target = "HD30501"       

print("Mass ratio ", HD30501_mass[0]/HD30501_mass[1])
star_rv = RV_from_params(HD30501_times, HD30501_params, use_offset=False)

planet_rv = RV_planet(HD30501_times, HD30501_params, HD30501_mass, use_offset=False)

print("star_rv   ", star_rv)
print("planet_rv ", planet_rv)




In [ ]:
# Need to test how the shift changes when only have the minimum mass for planet. 

In [ ]:
# Effect of I on MsinI

I = np.linspace(0.2,np.pi/2,100)[::-1]
MsinI = 50

M = MsinI/np.sin(I)

plt.figure()
plt.plot(I*180/np.pi, M)
plt.title("Mass verse Inclination, msinI=50 Mjup")
plt.ylabel("Planet Mass (MJup)")
plt.show()

In [ ]:
#Effect of msinI instead of M in K determination
K1 = 500   # m/s
m1 = 1000   # MJ
m2sinI = 50 # MJ
m2 = m2sinI/np.sin(I)  # MJ
K2 = K1 * -m1/m2

plt.figure()
plt.plot(I*180/np.pi, K2)
plt.xlabel("Inclination I (deg)")
plt.ylabel("Planet RV amplitude K2")
plt.title("Inclination on Planet RV Semi-Amplitude K2")
plt.show()

#K2 for planet is maximum at minimum mass

#### Testing Junk - no need to continue

In [ ]:
test_rv_curves(HD30501_times, HD30501_params)
test_rv_curves(HD162020_times, HD162020_params)
test_rv_curves(HD202206_times, HD202206b_params)
test_rv_curves(HD202206_times, HD202206c_params)
test_rv_curves(HD211847_times, HD211847_params)
test_rv_curves(HD4747_times, HD4747_params)
test_rv_curves(HD167665_times, HD167665_params)
test_rv_curves(HD168443_times, HD168443b_params)
test_rv_curves(HD168443_times, HD168443c_params)


In [ ]:
#%timeit aj_rv = pl_rv_array(HD30501_times, *HD30501_params) # *unpacks parameters from list
    
#%timeit py_rv = rv_curve_py(HD30501_times, *HD30501_params) # ajplanet version is 15 X faster 

In [ ]:
# Test RV determination 

# Get parameters for star and taget
Red_JD = 2400000
#hd30501
mean_val = 0 # 23.710 # +- 0.017
Period = 2073.6   # +3 - 2.9 days
e = 0.741         # +_ 0.04
K1 = 1703.1       #+- 26  ms^-1
Tau = 53851.5      # +- 3 JD    Note: I think this is tau but I am not sure. 
omega = 70.4 * np.pi/180.     # +- 0.7 deg
values = np.empty_like([1,1,1,1])

for i, ti in enumerate(np.array([2456024.505902, 2456140.887153, 2456141.866329, 2456145.904258]) - Red_JD):
    # Calculate Mean anomaly:
    M_anomaly = meananomaly(ti, Tau, Period)

    # Determine true anomaly from mean anomaly
    true_anomaly = trueanomaly(M_anomaly, e)

    # Now Calculalte the RV of Star
    RV = mean_val + K1 *(np.cos(true_anomaly + omega) + e * np.cos(omega))
    

    print("RV value {0} ms^-1 at time {1} JD*".format( RV ,  ti ))
    values[i] = RV
print(values)
print("Difference in RV between observations ", np.diff(values), " m/s,   --- These values don't match with above")  